In [2]:
"""
Created on Thu Jan 30 17:46:18 2025

@author: Thierry ALLEM


TRAITEMENT DE LA BASE DE DONNES SYNOP INCLUANT DES DONNEES METEOROLOGIQUES

"""

'\nCreated on Thu Jan 30 17:46:18 2025\n\n@author: Thierry ALLEM\n\n\nTRAITEMENT DE LA BASE DE DONNES SYNOP INCLUANT DES DONNEES METEOROLOGIQUES\n\n'

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Importation du fichier des relevés de températures
# Lecture du fichier CSV
df_meteo = pd.read_csv("donnees-synop-essentielles-omm.csv", sep=';', encoding='utf-8')
df_meteo.head()

,ID OMM station,Date,Pression au niveau mer,Variation de pression en 3 heures,Type de tendance barométrique,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Point de rosée,Humidité,...,Altitude,communes (name),communes (code),EPCI (name),EPCI (code),department (name),department (code),region (name),region (code),mois_de_l_annee
0,7558,2010-01-05T10:00:00+01:00,100280.0,-50.0,5.0,260.0,1.5,275.75,275.75,100.0,...,712,Millau,12145,CC de Millau Grands Causses,241200567.0,Aveyron,12,Occitanie,76.0,1
1,61976,2010-01-05T10:00:00+01:00,100990.0,NaN,NaN,NaN,NaN,305.45,299.05,69.0,...,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,7027,2010-01-05T13:00:00+01:00,100720.0,-190.0,8.0,200.0,3.6,273.65,271.75,87.0,...,67,Carpiquet,14137,CU Caen la Mer,200065597.0,Calvados,14,Normandie,28.0,1
3,7110,2010-01-05T13:00:00+01:00,100750.0,-230.0,8.0,210.0,4.1,276.95,272.55,73.0,...,94,Guipavas,29075,Brest Métropole,242900314.0,Finistère,29,Bretagne,53.0,1
4,7591,2010-01-05T13:00:00+01:00,NaN,NaN,NaN,NaN,NaN,274.45,269.05,67.0,...,871,Embrun,05046,CC Serre-Ponçon,200067742.0,Hautes-Alpes,05,Provence-Alpes-Côte d'Azur,93.0,1


In [5]:
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557951 entries, 0 to 2557950
Data columns (total 82 columns):
 #   Column                                                     Dtype  
---  ------                                                     -----  
 0   ID OMM station                                             int64  
 1   Date                                                       object 
 2   Pression au niveau mer                                     float64
 3   Variation de pression en 3 heures                          float64
 4   Type de tendance barométrique                              float64
 5   Direction du vent moyen 10 mn                              float64
 6   Vitesse du vent moyen 10 mn                                float64
 7   Température                                                float64
 8   Point de rosée                                             float64
 9   Humidité                                                   float64
 10  Visibilité horizon

In [6]:
# Renommage de colonnes
colonnes_rename = {
    'Date': 'date_heure',
    'Température': 'temperature_K',
    'Humidité': 'humidite_relative',
    'Température (°C)': 'temperature_C',
    'region (name)': 'region'}
df_meteo.rename(columns=colonnes_rename, inplace=True)
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557951 entries, 0 to 2557950
Data columns (total 82 columns):
 #   Column                                                     Dtype  
---  ------                                                     -----  
 0   ID OMM station                                             int64  
 1   date_heure                                                 object 
 2   Pression au niveau mer                                     float64
 3   Variation de pression en 3 heures                          float64
 4   Type de tendance barométrique                              float64
 5   Direction du vent moyen 10 mn                              float64
 6   Vitesse du vent moyen 10 mn                                float64
 7   temperature_K                                              float64
 8   Point de rosée                                             float64
 9   humidite_relative                                          float64
 10  Visibilité horizon

In [7]:
# Filtrage  et affichage de toutes les lignes avec des doublons en fonction des colonnes 'date_heure','region' et 'ID OMM station'
doublons_lignes = df_meteo[df_meteo.duplicated(subset=['date_heure', 'region','ID OMM station'], keep=False)]

# Affichage des doublons avec toutes leurs colonnes
print(doublons_lignes)

         ID OMM station                 date_heure  Pression au niveau mer  \
88109              7471  2016-12-21T16:00:00+01:00                     NaN   
152318             7280  2016-05-06T14:00:00+02:00                101290.0   
237752             7510  2015-03-29T08:00:00+02:00                102480.0   
412763             7280  2016-05-06T14:00:00+02:00                101290.0   
555913             7471  2016-12-20T13:00:00+01:00                     NaN   
785539             7471  2016-12-21T16:00:00+01:00                     NaN   
874912             7510  2015-03-29T08:00:00+02:00                102480.0   
896432             7650  2011-01-12T13:00:00+01:00                102170.0   
949487             7471  2016-12-20T13:00:00+01:00                     NaN   
1116732            7650  2011-01-12T13:00:00+01:00                102170.0   

         Variation de pression en 3 heures  Type de tendance barométrique  \
88109                                -20.0                      

In [8]:
# Suppression des doublons en conservant seulement la première occurrence
df_meteo = df_meteo.drop_duplicates(subset=['date_heure', 'region', 'ID OMM station'], keep='first')

In [9]:
# Filtrage  et affichage de toutes les lignes avec des doublons en fonction des colonnes 'date_heure','region' et 'ID OMM station'
doublons_lignes2 = df_meteo[df_meteo.duplicated(subset=['date_heure', 'region','ID OMM station'], keep=False)]

# Affichage des doublons avec toutes leurs colonnes
print(doublons_lignes2)

Empty DataFrame
Columns: [ID OMM station, date_heure, Pression au niveau mer, Variation de pression en 3 heures, Type de tendance barométrique, Direction du vent moyen 10 mn, Vitesse du vent moyen 10 mn, temperature_K, Point de rosée, humidite_relative, Visibilité horizontale, Temps présent, Temps passé 1, Temps passé 2, Nebulosité totale, Nébulosité  des nuages de l' étage inférieur, Hauteur de la base des nuages de l'étage inférieur, Type des nuages de l'étage inférieur, Type des nuages de l'étage moyen, Type des nuages de l'étage supérieur, Pression station, Niveau barométrique, Géopotentiel, Variation de pression en 24 heures, Température minimale sur 12 heures, Température minimale sur 24 heures, Température maximale sur 12 heures, Température maximale sur 24 heures, Température minimale du sol sur 12 heures, Méthode de mesure Température du thermomètre mouillé, Température du thermomètre mouillé, Rafale sur les 10 dernières minutes, Rafales sur une période, Periode de mesure de l

In [10]:
# Suppression de colonnes non utiles dans cette étude
colonnes_to_drop_humidite = [
    0, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 
    24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 
    42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 
    60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78
]
df_meteo = df_meteo.drop(df_meteo.columns[colonnes_to_drop_humidite], axis=1)
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2557946 entries, 0 to 2557950
Data columns (total 7 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   date_heure                   object 
 1   Vitesse du vent moyen 10 mn  float64
 2   humidite_relative            float64
 3   temperature_C                float64
 4   region                       object 
 5   region (code)                float64
 6   mois_de_l_annee              int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 156.1+ MB


In [11]:
df_meteo['region'].unique()

array(['Occitanie', nan, 'Normandie', 'Bretagne',
       "Provence-Alpes-Côte d'Azur", 'Martinique', 'Hauts-de-France',
       'Île-de-France', 'Guyane', 'Corse', 'Grand Est',
       'Nouvelle-Aquitaine', 'Bourgogne-Franche-Comté',
       'Pays de la Loire', 'Auvergne-Rhône-Alpes', 'La Réunion',
       'Centre-Val de Loire', 'Guadeloupe', 'Mayotte',
       'Terres australes et antarctiques françaises', 'Saint-Barthélemy',
       'Saint-Pierre-et-Miquelon'], dtype=object)

In [12]:
# Suppression des lignes de régions non concernées par cette étude
regions_a_exclure = [
    np.nan, 'Martinique', 'Guyane', 'Corse', 'La Réunion', 'Guadeloupe', 
    'Mayotte', 'Saint-Barthélemy', 'Terres australes et antarctiques françaises',
    'Saint-Pierre-et-Miquelon'
]
df_meteo = df_meteo[~df_meteo['region'].isin(regions_a_exclure)]
df_meteo['region'].unique()

array(['Occitanie', 'Normandie', 'Bretagne', "Provence-Alpes-Côte d'Azur",
       'Hauts-de-France', 'Île-de-France', 'Grand Est',
       'Nouvelle-Aquitaine', 'Bourgogne-Franche-Comté',
       'Pays de la Loire', 'Auvergne-Rhône-Alpes', 'Centre-Val de Loire'],
      dtype=object)

In [13]:
# Renommage des valeurs de 'region'
renommage_regions = {
    'Occitanie': 'OCCITANIE',
    'Normandie': 'NORMANDIE',
    'Bretagne': 'BRETAGNE',
    'Provence-Alpes-Côte d\'Azur': 'PROVENCE ALPES COTE D AZUR',
    'Hauts-de-France': 'HAUTS DE FRANCE',
    'Île-de-France': 'ILE DE FRANCE',
    'Grand Est': 'GRAND EST',
    'Nouvelle-Aquitaine': 'NOUVELLE AQUITAINE',
    'Bourgogne-Franche-Comté': 'BOURGOGNE FRANCHE COMTE',
    'Pays de la Loire': 'PAYS DE LA LOIRE',
    'Auvergne-Rhône-Alpes': 'AUVERGNE RHONE ALPES',
    'Centre-Val de Loire': 'CENTRE VAL DE LOIRE'
}
df_meteo['region'] = df_meteo['region'].replace(renommage_regions)
df_meteo['region'].unique()

array(['OCCITANIE', 'NORMANDIE', 'BRETAGNE', 'PROVENCE ALPES COTE D AZUR',
       'HAUTS DE FRANCE', 'ILE DE FRANCE', 'GRAND EST',
       'NOUVELLE AQUITAINE', 'BOURGOGNE FRANCHE COMTE',
       'PAYS DE LA LOIRE', 'AUVERGNE RHONE ALPES', 'CENTRE VAL DE LOIRE'],
      dtype=object)

In [14]:
# Conversion des dates avec gestion des fuseaux horaires
df_meteo["date_heure"] = pd.to_datetime(df_meteo["date_heure"], errors='coerce', utc=True)

# Conversion à l'heure française
df_meteo["date_heure"] = df_meteo["date_heure"].dt.tz_convert("Europe/Paris")

# Suppression de la mention de fuseau tout en conservant l'heure correcte
df_meteo["date_heure"] = df_meteo["date_heure"].dt.tz_localize(None)

# Extraction de l'année
df_meteo['annee'] = df_meteo['date_heure'].dt.year

# Ajout d'une colonne 'date' avec le format 'yyyy-mm-dd'
df_meteo['date'] = df_meteo['date_heure'].dt.strftime('%Y-%m-%d')

# Ajout d'une colonne 'heure' à l'heure française
df_meteo['heure'] = df_meteo['date_heure'].dt.strftime('%H:%M')

df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1741602 entries, 0 to 2557950
Data columns (total 10 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   date_heure                   datetime64[ns]
 1   Vitesse du vent moyen 10 mn  float64       
 2   humidite_relative            float64       
 3   temperature_C                float64       
 4   region                       object        
 5   region (code)                float64       
 6   mois_de_l_annee              int64         
 7   annee                        int32         
 8   date                         object        
 9   heure                        object        
dtypes: datetime64[ns](1), float64(4), int32(1), int64(1), object(3)
memory usage: 139.5+ MB


In [15]:
# Suppression des valeurs NaT après conversion
df_meteo = df_meteo.dropna(subset=["date_heure"])

In [16]:
# Filtrage des années pour limiter le dataset de 2013 à 2022 inclus
df_meteo = df_meteo[(df_meteo["annee"] >= 2013) & (df_meteo["annee"] <= 2022)]

# Vérification
print(df_meteo['annee'].unique())

[2013 2015 2014 2016 2017 2018 2019 2020 2022 2021]


In [17]:
# recherche des valeurs manquantes
df_meteo.isna().sum()

date_heure                         0
Vitesse du vent moyen 10 mn     3117
humidite_relative              10525
temperature_C                   9768
region                             0
region (code)                      0
mois_de_l_annee                    0
annee                              0
date                               0
heure                              0
dtype: int64

In [18]:
# Remplacement des NaN par 0 dans tout le dataframe
df_meteo.fillna(0, inplace=True)
df_meteo.isna().sum()

date_heure                     0
Vitesse du vent moyen 10 mn    0
humidite_relative              0
temperature_C                  0
region                         0
region (code)                  0
mois_de_l_annee                0
annee                          0
date                           0
heure                          0
dtype: int64

In [19]:
# ****************  Ajout des températures ressenties sous l'effet du vent et de l'humidité *****

In [20]:
# Fonction
def calcul_temperature_ressentie(df):
    # Conversion vitesse vent m/s en km/h
    df['vent_kmh'] = df['Vitesse du vent moyen 10 mn'] * 3.6
    
    # Formule du refroidissement éolien (Wind Chill Index)
    wind_chill = (
        13.12 
        + 0.6215 * df['temperature_C'] 
        - 11.37 * np.power(df['vent_kmh'], 0.16) 
        + 0.3965 * df['temperature_C'] * np.power(df['vent_kmh'], 0.16)
    )
    
    # Formule du Heat Index (simplifiée)
    hi_coeffs = [ 
        -8.78469475556, 1.61139411, 2.33854883889, -0.14611605,
        -0.012308094, -0.0164248277778, 0.002211732, 0.00072546, -0.000003582
    ]
    T = df['temperature_C']
    RH = df['humidite_relative']
    
    heat_index = (
        hi_coeffs[0] 
        + hi_coeffs[1] * T 
        + hi_coeffs[2] * RH 
        + hi_coeffs[3] * T * RH 
        + hi_coeffs[4] * T**2 
        + hi_coeffs[5] * RH**2 
        + hi_coeffs[6] * T**2 * RH 
        + hi_coeffs[7] * T * RH**2 
        + hi_coeffs[8] * T**2 * RH**2
    )
    
    # Conditions d'application
    conditions = [
        (df['temperature_C'] <= 10) & (df['vent_kmh'] > 4.8),  # Wind Chill
        (df['temperature_C'] >= 27) & (df['humidite_relative'] > 40),  # Heat Index
    ]
    choices = [wind_chill, heat_index]
    
    # Valeur ressentie par défaut = température réelle
    df['temperature_ressentie'] = np.select(conditions, choices, default=df['temperature_C'])
    
    return df.drop(columns=['vent_kmh'])

In [21]:
# Application de la fonction
df_meteo = calcul_temperature_ressentie(df_meteo)

In [22]:
# Suppression de la colonne 'humidite_relative'
df_meteo = df_meteo.drop(columns =['humidite_relative'])
df_meteo.head()

,date_heure,Vitesse du vent moyen 10 mn,temperature_C,region,region (code),mois_de_l_annee,annee,date,heure,temperature_ressentie
1788,2013-03-05 04:00:00,14.9,11.8,OCCITANIE,76.0,3,2013,2013-03-05,04:00,11.800000
1790,2013-03-05 07:00:00,1.5,2.8,NORMANDIE,28.0,3,2013,2013-03-05,07:00,1.422594
1791,2013-03-05 07:00:00,0.0,-3.0,GRAND EST,44.0,3,2013,2013-03-05,07:00,-3.000000
1792,2013-03-05 07:00:00,10.3,10.0,AUVERGNE RHONE ALPES,84.0,3,2013,2013-03-05,07:00,6.134590
1793,2013-03-05 07:00:00,13.9,11.1,OCCITANIE,76.0,3,2013,2013-03-05,07:00,11.100000


In [23]:
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1159540 entries, 1788 to 2557459
Data columns (total 10 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   date_heure                   1159540 non-null  datetime64[ns]
 1   Vitesse du vent moyen 10 mn  1159540 non-null  float64       
 2   temperature_C                1159540 non-null  float64       
 3   region                       1159540 non-null  object        
 4   region (code)                1159540 non-null  float64       
 5   mois_de_l_annee              1159540 non-null  int64         
 6   annee                        1159540 non-null  int32         
 7   date                         1159540 non-null  object        
 8   heure                        1159540 non-null  object        
 9   temperature_ressentie        1159540 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int32(1), int64(1), object(3)
memory usage: 92

In [24]:
# Export
df_meteo.to_csv('df_temperature_ressentie.csv', index=False)
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1159540 entries, 1788 to 2557459
Data columns (total 10 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   date_heure                   1159540 non-null  datetime64[ns]
 1   Vitesse du vent moyen 10 mn  1159540 non-null  float64       
 2   temperature_C                1159540 non-null  float64       
 3   region                       1159540 non-null  object        
 4   region (code)                1159540 non-null  float64       
 5   mois_de_l_annee              1159540 non-null  int64         
 6   annee                        1159540 non-null  int32         
 7   date                         1159540 non-null  object        
 8   heure                        1159540 non-null  object        
 9   temperature_ressentie        1159540 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int32(1), int64(1), object(3)
memory usage: 92

In [25]:
# ********************  INTERPOLATION DES DONNEES METEO POUR OBTENIR DES DONNEES PAR PAS DE 30 MINUTES  ************************
# ******************** CALCUL DES MOYENNES ET INTERPOLATION DES DONNÉES ************************

In [26]:
# Colonnes à interpoler
columns_to_interpolate = ['Vitesse du vent moyen 10 mn', 'temperature_C', 'temperature_ressentie']

# Calcul des moyennes pour chaque combinaison de date_heure et région
df_meteo_grouped = df_meteo.groupby(["date_heure", "region"], as_index=False).mean(numeric_only=True)

# Création de la plage de dates avec un pas de 30 minutes
date_range = pd.date_range(
    start="2013-01-01 00:00:00",
    end="2022-12-31 23:00:00",
    freq="30min")

# Création du DataFrame de référence
regions = df_meteo['region'].unique()
all_dates = pd.DataFrame({
    "date_heure": np.tile(date_range, len(regions)),
    "region": np.repeat(regions, len(date_range))
})

# Jointure pour aligner les dates avec df_meteo_grouped
df_meteo_aligned = pd.merge(
    all_dates, df_meteo_grouped,
    on=["date_heure", "region"],
    how="left"
)

# Interpolation par région avec affichage d'une barre de progression
tqdm.pandas(desc="Interpolation des données")

# Interpolation des colonnes sélectionnées
interpolated_df = df_meteo_aligned.groupby("region").progress_apply(
    lambda group: group.set_index("date_heure")[columns_to_interpolate]
    .interpolate(method="time", limit_direction="both")
    .reset_index()
)

# Fusion avec les autres colonnes
interpolated_df = pd.merge(
    df_meteo_aligned.drop(columns=columns_to_interpolate),
    interpolated_df,
    on=["date_heure", "region"],
    how="left"
)

# Suppression de colonnes non utiles
colonnes_to_drop_interpolated =['region (code)','mois_de_l_annee','annee']
interpolated_df=interpolated_df.drop(colonnes_to_drop_interpolated, axis = 1)


# Tri et sauvegarde des résultats
df_interpolated = interpolated_df.sort_values(['region', 'date_heure'])
df_interpolated.to_csv("df_meteo_interpolated.csv", index=False)

Interpolation des données: 100%|██████████| 12/12 [00:00<00:00, 13.96it/s]


In [27]:
df_interpolated.head()

,date_heure,region,Vitesse du vent moyen 10 mn,temperature_C,temperature_ressentie
525885,2013-01-01 00:00:00,AUVERGNE RHONE ALPES,8.475000,7.375000,3.347231
525886,2013-01-01 00:30:00,AUVERGNE RHONE ALPES,8.475000,7.375000,3.347231
525887,2013-01-01 01:00:00,AUVERGNE RHONE ALPES,8.475000,7.375000,3.347231
525888,2013-01-01 01:30:00,AUVERGNE RHONE ALPES,8.154167,7.416667,3.548545
525889,2013-01-01 02:00:00,AUVERGNE RHONE ALPES,7.833333,7.458333,3.749860


In [28]:
df_interpolated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2103540 entries, 525885 to 1227064
Data columns (total 5 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   date_heure                   datetime64[ns]
 1   region                       object        
 2   Vitesse du vent moyen 10 mn  float64       
 3   temperature_C                float64       
 4   temperature_ressentie        float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 96.3+ MB


In [31]:
print("Valeurs manquantes:",df_interpolated.isna().sum())

Valeurs manquantes: date_heure                     0
region                         0
Vitesse du vent moyen 10 mn    0
temperature_C                  0
temperature_ressentie          0
dtype: int64
